In [ ]:
import pandas as pd
import networkx as nx
from tqdm import tqdm 
import unittest
import matplotlib.pyplot as plt
%matplotlib inline
FGA_R = 2

def append_to_csv_file(filename, line):
    line = [str(l) for l in line]
    ','.join(line)
    with open(filename, 'a') as f:
        f.write(','.join(line) + '\n')


In [ ]:
def relabel_directed_graph_labels(labels):
    relabelled = {}
    for src, dst in labels:
        if (dst,src) not in relabelled:
            relabelled[(src,dst)] = str(labels[(src,dst)])
        else:
            relabelled[(dst, src)] = '{},{}'.format(relabelled[(dst, src)], str(labels[(src,dst)]))
    return relabelled

def draw_graph(gr):
    pos=nx.spring_layout(gr)
    nx.draw(gr,pos, with_labels=True)
    labels = nx.get_edge_attributes(gr,'weight')
    labels = relabel_directed_graph_labels(labels)
    nx.draw_networkx_edge_labels(gr,pos,edge_labels=labels)
    nx.draw_networkx_nodes(gr,pos)
    
def create_digraph(nodes, edges):
    gr = nx.DiGraph()
    gr.add_nodes_from(nodes)
    gr.add_edges_from([(x,y) for x,y,z in edges])
    for x,y,weight in edges:
        gr[x][y]['weight'] = weight
    return gr

In [ ]:
def FGA(gr, iters = 20):
    fairness = dict(zip(gr.nodes, len(gr.nodes)*[1]))
    goodness = dict(zip(gr.nodes, len(gr.nodes)*[1]))
    for i in range(iters):
        new_fairness = {}
        new_goodness = {}
        for v in gr.nodes:
            if gr.in_degree(v) == 0:
                new_goodness[v] = 1 # TODO discuss this condition
                continue
            new_goodness[v] = sum([fairness[x]*gr[x][y]['weight'] for x,y in gr.in_edges(v)])/gr.in_degree(v)

        for v in gr.nodes:
            if gr.out_degree(v) == 0:
                new_fairness[v] = 1 # TODO discuss this condition
                continue
            new_fairness[v] = 1 - sum([abs(gr[x][y]['weight']-new_goodness[y]) for x,y in gr.out_edges(v)])/(FGA_R*gr.out_degree(v))
        fairness = new_fairness
        goodness = new_goodness
    nx.set_node_attributes(gr, fairness, 'fairness')
    nx.set_node_attributes(gr, goodness, 'goodness')
    return fairness, goodness

In [ ]:
# GET DATA

ots = pd.read_csv("./data-wsn/db/OTCNet.csv", header = None)
gr = nx.DiGraph()
gr.add_nodes_from(ots[0].unique())
gr.add_nodes_from(ots[1].unique())
for i,row in tqdm(ots.iterrows()):
    gr.add_edge(row[0], row[1], weight = row[2])



In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

def perform_direct_attack(gr_exp, K, T, A, iters):
    t = T[0]
    print("Target before", g[t], 'indeg:', len(gr.in_edges(t)))
    new_edges = [(v, t) for v in A]
    gr_exp.add_edges_from(new_edges)
    for v in A:
        gr_exp[v][t]['weight'] = -1 # (minimize the reputation)
    f1,g1 = FGA(gr_exp, iters)
    print("Target after", g1[t])
    return [K, T, iters, len(gr.in_edges(t)), g[t], g1[t]]
    

def attack_direct_grounded(gr, K, T, iters, fga):
    f,g = fga
    print('Direct Grounded Attack')
    well_grounded = [n for n in gr.nodes if f[n] > 0.7 and len(gr.out_edges(n)) > 5]
    A = random.sample(set(well_grounded).difference(set(T)), K)
    print('Attackers', A, 'Target', T)
    gr_exp = gr.copy()
    params = perform_direct_attack(gr_exp, K, T, A, iters, )
    return append_to_csv_file('DG.csv', params)
    

def attack_direct_not_grounded(gr, K, T, iters, fga):
    f,g = fga
    print('Direct NOT Grounded Attack')
    A = list(range(max(gr.nodes) + 1, max(gr.nodes)+ K + 1))
    print('Attackers', A, 'Target', T)
    gr_exp = gr.copy()
    gr_exp.add_nodes_from(A)
    params = perform_direct_attack(gr_exp, K, T, A, iters)
    return append_to_csv_file('DNG.csv', params)
    
    
def find_best_greedy(gr_exp, S, t, fga, iters):
    f,g = fga
    best = -1
    strategy = None
    edge = None
    print('Find best greedy')
    for e in tqdm(S):
        for weight in [-1,1]:
            gr_exp_copy = gr_exp.copy()
            gr_exp_copy.add_edges_from([e,])
            gr_exp_copy[e[0]][e[1]]['weight'] = weight
            f1,g1 = FGA(gr_exp_copy, iters)
            if g1[t] < g[t] and abs(g1[t]-g[t]) > best:
                strategy = weight
                best = abs(g1[t]-g[t])
                edge = e
    return edge, strategy

def find_neighbours(gr, t, dist = 2):
    if dist == 2:
        neighbours1 = [e[0] for e in gr.in_edges(t)]
        neighbours2 = flatten([gr.out_edges(e) for e in neighbours1])
        neighbours2 = [int(n[1]) for n in neighbours2 if n[1] != t]
        return neighbours2
    

def perform_dist_attack(gr_exp, dist, iters, K, T, A, fga):
    f,g = fga
    t = T[0]
    print("Target before", g[t], 'indeg:', len(gr.in_edges(t)))
    neighbours = find_neighbours(gr_exp, t, dist)
    print("Num of neighbours to attack", len(neighbours))
    for a in sorted(A, key=lambda x:f[x]):
        edge, weight = find_best_greedy(gr_exp, [(a,n) for n in neighbours], t, fga, iters)
        if edge is None:
            print("Best greedy returned None")
            break
        gr_exp.add_edges_from([edge])
        gr_exp[edge[0]][edge[1]]['weight'] = weight
        f,g = fga
        print('Edge, strategy', edge, weight)

    f1,g1 = FGA(gr_exp, iters)
    print("Target after", g1[t])
    return [K, T, iters, len(gr.in_edges(t)), g[t], g1[t], len(neighbours)]
    
def attack_mixed(gr, K1, K2, dist, T, iters, fga):
    f,g = fga
    print('Mixed Attack', K1, K2)
    A1 = list(range(max(gr.nodes) + 1, max(gr.nodes)+ K1 + 1))
    A2 = list(range(max(gr.nodes) + K1 + 1, max(gr.nodes)+ K1 + K2 + 1))
    print('Attackers', A1, A2, 'Target', T)
    gr_exp = gr.copy()
    gr_exp.add_nodes_from(A1)
    gr_exp.add_nodes_from(A2)
    f,g = FGA(gr_exp, iters)
    params1 = perform_direct_attack(gr_exp, K1, T, A1, iters)
    params2 = perform_dist_attack(gr_exp, dist, iters, K2, T, A2, (f,g))
    
    append_to_csv_file('MIX.csv', [K1, K2, T, iters, params1[3], params1[4], params1[5], params2[5], params2[6]])
    
    
    
def attack_dist_grounded(gr, K, T, dist, iters, fga):
#     append_to_csv_file('STG.csv', ['K', 'T', 'iters', 'INDEG', 'START', 'END', 'NEIGHBOURS'])
    f,g = fga
    print('DIST Grounded Attack')
    well_grounded = [n for n in gr.nodes if f[n] > 0.7 and len(gr.out_edges(n)) > 5]
    print(len(well_grounded))
    A = random.sample(set(well_grounded).difference(set(T)), K)
    print('Attackers', A, 'Target', T)
    gr_exp = gr.copy()
    params = perform_dist_attack(gr_exp, dist, iters, K, T, A, fga)
    append_to_csv_file('STG.csv', params)
    

def attack_dist_not_grounded(gr, K, T, dist, iters, fga):
#     append_to_csv_file('STNG.csv', ['K', 'T', 'iters', 'INDEG', 'START', 'END', 'NEIGHBOURS'])
    f,g = fga
    print('Direct NOT Grounded Attack')
    A = list(range(max(gr.nodes) + 1, max(gr.nodes)+ 1+ K))
    print('Attackers', A, 'Target', T)
    gr_exp = gr.copy()
    gr_exp.add_nodes_from(A)
    f,g = FGA(gr_exp, iters)
    params = perform_dist_attack(gr_exp, dist, iters, K, T, A, (f,g))
    append_to_csv_file('STNG.csv', params)
    
    


    
    


In [ ]:
import random

ITERS = 4
f,g = FGA(gr, ITERS)

NUM_EXPERIMENTS = 25
for i in range(NUM_EXPERIMENTS):
    for K in range(1,8):
        target_candidates = [n for n in gr.nodes if g[n] > 0.5 and len(gr.in_edges(n)) < 10 and len(gr.in_edges(n)) > 0]
        T = random.sample(set(target_candidates), 1)


        attack_direct_grounded(gr, K, T, ITERS, (f,g))
        attack_direct_not_grounded(gr, K, T, ITERS, (f,g))
        attack_dist_grounded(gr, K, T, 2, ITERS, (f,g))
        attack_dist_not_grounded(gr, K, T, 2, ITERS, (f,g))


In [ ]:
import random

ITERS = 4
# K = 1
f,g = FGA(gr, ITERS)

NUM_EXPERIMENTS = 15
for i in range(NUM_EXPERIMENTS):
    for K in range(2,8):
        for K1 in range(1, K):
            K2 = K - K1
            target_candidates = [n for n in gr.nodes if g[n] > 0.5 and len(gr.in_edges(n)) < 10 and len(gr.in_edges(n)) > 0]
            T = random.sample(set(target_candidates), 1)

            attack_mixed(gr, K1, K2, 2, T, ITERS, (f,g))


In [ ]:
MAX_ATTACK_SIZE = 10
def perform_better_dist_attack(gr_exp, dist, iters, K, T, A, fga):
    f,g = fga
    t = T[0]
    print("Target before", g[t], 'indeg:', len(gr.in_edges(t)))
    neighbours = find_neighbours(gr_exp, t, dist)
    print("Num of neighbours to attack", len(neighbours))
    A = list(sorted(A, key=lambda x:f[x]))
    i = 0
    while i < len(A):
        a = A[i]
        edge0, weight = find_best_greedy(gr_exp, [(a,n) for n in neighbours], t, fga, iters)
        if edge0 is None:
            print("Best greedy returned None")
            break
        edges_len = min(5*len(gr_exp.in_edges(edge0[1])), MAX_ATTACK_SIZE, len(A)-i)
        print('Edges len', edges_len)
        edges = zip(A[i:i+edges_len], [edge0[1]]*edges_len)
        i += edges_len
        for edge in edges:
            gr_exp.add_edges_from([edge])
            gr_exp[edge[0]][edge[1]]['weight'] = weight
            print('Edge, strategy', edge, weight)
        f1,g1 = FGA(gr_exp, iters)
        print("Target after", g1[t])
    return [K, T, iters, len(gr.in_edges(t)), g[t], g1[t], len(neighbours)]

    def attack_better_dist_grounded(gr, K, T, dist, iters, fga):
    # append_to_csv_file('NEWDIST.csv', ['K', 'T', 'iters', 'INDEG', 'START', 'END', 'NEIGHBOURS'])
    f,g = fga
    print('DIST Better Grounded Attack')
    well_grounded = [n for n in gr.nodes if f[n] > 0.7 and len(gr.out_edges(n)) > 5]
    A = random.sample(set(well_grounded).difference(set(T)), K)
    print('Attackers', A, 'Target', T)
    gr_exp = gr.copy()
    params = perform_better_dist_attack(gr_exp, dist, iters, K, T, A, fga)
    append_to_csv_file(name + '_NEWDIST.csv', params)

In [ ]:
from itertools import chain

ITERS = 4
f,g = FGA(gr, ITERS)

NUM_EXPERIMENTS = 20
for i in range(NUM_EXPERIMENTS):
    K = 20
    cond = False
    target = None
    while not cond:
        target = random.choice(list(gr.nodes()))
        l0 = map(lambda e: gr.out_edges(e[0]), gr.in_edges(target))
        l1 = list(chain(*l0))
        l2 = list(map(lambda e: int(e[1]), l1))
        cond = any([g[x] > 0.5 or g[x] < -0.5 for x in l2]) and len(gr.in_edges(target)) > 1 and len(gr.in_edges(target)) < 10 and g[target] > 0.5
    attack_better_dist_grounded(gr, K, [target], 2, ITERS, (f,g))